<a href="https://colab.research.google.com/github/karthikeyanbu/GenAI/blob/main/Karthikeyan_GenAI_04Feb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
## About the program
# Download customer dataset from Kaggle and perform text pre-processing - data cleansing steps and encoding steps
# 1. Converting to lowercase
# 2. Remove punctuations
# 3. Remove URL, if any
# 4. Remove stopwords
# 5. Remove html tags, if any
# 6. Remove emoji, if any
# 7. Spelling correction
# 8. Abbrevation expansion
# 9. Splitting
# 10. Stemming
# 11. Lemmetization
# 12. Encoding - Bag of words
# 13. Bigram and Trigram
# 14. TF IDF

In [2]:
# Installations
!apt-get install wget
!wget https://raw.githubusercontent.com/karthikeyanbu/GenAI/main/twcs.csv

!pip install emoji
import nltk
nltk.download("punkt")
nltk.download('wordnet')

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
wget is already the newest version (1.21.2-2ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 32 not upgraded.
--2024-02-07 09:45:06--  https://raw.githubusercontent.com/karthikeyanbu/GenAI/main/twcs.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20787426 (20M) [text/plain]
Saving to: ‘twcs.csv.1’

twcs.csv.1          100%[===================>]  19.82M   131MB/s    in 0.2s    

2024-02-07 09:45:06 (131 MB/s) - ‘twcs.csv.1’ saved [20787426/20787426]



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [70]:
# Libraries
import pandas as pd
import string as str
import re
import nltk
from nltk.corpus import stopwords
import emoji
import textblob
from textblob import TextBlob
from nltk.tokenize import word_tokenize,sent_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [4]:
# Read csv file
# Low_memory = False parameter set as per of debug message
customer_df = pd.read_csv("twcs.csv", low_memory=False)

In [5]:
customer_df.head()

,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id
0,1.0,sprintcare,False,Tue Oct 31 22:10:47 +0000 2017,@115712 I understand. I would like to assist y...,2,3.0
1,2.0,115712,True,Tue Oct 31 22:11:45 +0000 2017,@sprintcare and how do you propose we do that,NaN,1.0
2,3.0,115712,True,Tue Oct 31 22:08:27 +0000 2017,@sprintcare I have sent several private messag...,1,4.0
3,4.0,sprintcare,False,Tue Oct 31 21:54:49 +0000 2017,@115712 Please send us a Private Message so th...,3,5.0
4,5.0,115712,True,Tue Oct 31 21:49:35 +0000 2017,@sprintcare I did.,4,6.0


In [6]:
customer_df.shape

(152758, 7)

In [7]:
customer_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 152758 entries, 0 to 152757
Data columns (total 7 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   tweet_id                 109227 non-null  float64
 1   author_id                109227 non-null  object 
 2   inbound                  109227 non-null  object 
 3   created_at               109227 non-null  object 
 4   text                     109227 non-null  object 
 5   response_tweet_id        73904 non-null   object 
 6   in_response_to_tweet_id  81056 non-null   float64
dtypes: float64(2), object(5)
memory usage: 8.2+ MB


In [8]:
demo_df = customer_df[0:200]
demo_df

,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id
0,1.0,sprintcare,False,Tue Oct 31 22:10:47 +0000 2017,@115712 I understand. I would like to assist y...,2,3.0
1,2.0,115712,True,Tue Oct 31 22:11:45 +0000 2017,@sprintcare and how do you propose we do that,NaN,1.0
2,3.0,115712,True,Tue Oct 31 22:08:27 +0000 2017,@sprintcare I have sent several private messag...,1,4.0
3,4.0,sprintcare,False,Tue Oct 31 21:54:49 +0000 2017,@115712 Please send us a Private Message so th...,3,5.0
4,5.0,115712,True,Tue Oct 31 21:49:35 +0000 2017,@sprintcare I did.,4,6.0
...,...,...,...,...,...,...,...
195,283.0,115774,True,Sat Oct 28 13:37:52 +0000 2017,@115769 are there any plans to add a fast crop...,282,NaN
196,284.0,AdobeCare,False,Tue Oct 31 21:54:16 +0000 2017,"@115775 Thanks for sharing the file, we'll get...",NaN,285.0
197,285.0,115775,True,Tue Oct 31 20:16:08 +0000 2017,@AdobeCare @115776 It’s intermittent for me. T...,"2,84,286",287.0
198,287.0,AdobeCare,False,Thu Oct 26 19:03:22 +0000 2017,"@115776 Unfortunately, we are not able to repr...",285,288.0


In [9]:
## text is the field taken for data cleansing
# 1. Converting to lowercase
demo_df["text"]
demo_df["text"] = demo_df["text"].str.lower()
demo_df["text"]

<ipython-input-9-9d2d155a8531>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  demo_df["text"] = demo_df["text"].str.lower()


0      @115712 i understand. i would like to assist y...
1          @sprintcare and how do you propose we do that
2      @sprintcare i have sent several private messag...
3      @115712 please send us a private message so th...
4                                     @sprintcare i did.
                             ...                        
195    @115769 are there any plans to add a fast crop...
196    @115775 thanks for sharing the file, we'll get...
197    @adobecare @115776 it’s intermittent for me. t...
198    @115776 unfortunately, we are not able to repr...
199    @adobecare seems pretty widespread: https://t....
Name: text, Length: 200, dtype: object

In [10]:
# Check for the presence of punctuation
demo_df["text"].dtypes
#demo_df["text"] = demo_df["text"].to_string()

demo_df
for i in demo_df["text"]:
  #if ("." in i):
     print (i)

@115712 i understand. i would like to assist you. we would need to get you into a private secured link to further assist.
@sprintcare and how do you propose we do that
@sprintcare i have sent several private messages and no one is responding as usual
@115712 please send us a private message so that we can further assist you. just click ‘message’ at the top of your profile.
@sprintcare i did.
@115712 can you please send us a private message, so that i can gain further details about your account?
@sprintcare is the worst customer service
@115713 this is saddening to hear. please shoot us a dm, so that we can look into this for you. -kc
@sprintcare you gonna magically change your connectivity for me and my whole family ? 🤥 💯
@115713 we understand your concerns and we'd like for you to please send us a direct message, so that we can further assist you. -aa
@sprintcare since i signed up with you....since day 1
@115713 h there! we'd definitely like to work with you on this, how long have you

In [11]:
# Check for the presence of html tags
for i in demo_df["text"]:
  if ("https:" in i):
     print (i)

@115714 whenever i contact customer support, they tell me i have shortcode enabled on my account, but i have never in the 4 years i've tried https://t.co/0g98rtnxpk
@115716 our department is part of the corporate office.  if you're particular area has gone to this format, we were unawa... https://t.co/p7xcmtzpqj
@115716 the information pertaining to the account assumption is correct.  this does need to be done at a local outlet wit... https://t.co/p7xcmtzpqj
actually that's a broken link you sent me and incorrect information https://t.co/v4yfrhr8vi
my picture on @ask_spectrum pretty much every day. why should i pay $171 per month? https://t.co/u6ptkqa5ik
@115722 md. and this was sent to the wrong address https://t.co/dmq1wzxjok
@chipotletweets @28 
i don't fit in my veggie burrito costume #halloween https://t.co/7tjdvpzlwn
@115729 i'm so sorry about that. please tell us more so we can help: https://t.co/ax7w1dx3y9 -becky
@115732 that's incredibly concerning. please tell us more here: h

In [12]:
# 2. Remove punctuations
exclude = type(str.punctuation)

In [13]:
def remove_punc(text):
    if not isinstance(text, exclude):
        return ''
    return re.sub(r'[^\w\s]', '', text)

In [14]:
demo_df["text"] = demo_df["text"].apply(remove_punc)
demo_df["text"].head(10)

<ipython-input-14-26552bd5c4ee>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  demo_df["text"] = demo_df["text"].apply(remove_punc)


0    115712 i understand i would like to assist you...
1         sprintcare and how do you propose we do that
2    sprintcare i have sent several private message...
3    115712 please send us a private message so tha...
4                                     sprintcare i did
5    115712 can you please send us a private messag...
6             sprintcare is the worst customer service
7    115713 this is saddening to hear please shoot ...
8    sprintcare you gonna magically change your con...
9    115713 we understand your concerns and wed lik...
Name: text, dtype: object

In [15]:
# 3. Remove URL, if any
def remove_url(text):
    pattern=re.compile(r'https?://\S+|www\.\S+')
    return pattern.sub("",text)

In [16]:
demo_df["text"] = demo_df["text"].apply(remove_url)

for i in demo_df["text"]:
  if ("https:" in i):
     print (i)

<ipython-input-16-0e09b11b2e2e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  demo_df["text"] = demo_df["text"].apply(remove_url)


In [17]:
# 4. Remove stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [18]:
def remove_stopwords(text):
    new_text=[]
    for word in text.split():
        if word in stopwords.words("english"):
            new_text.append("")
        else:
            new_text.append(word.strip())

    return " ".join(new_text).replace("   ","")

In [19]:
demo_df["text"] = demo_df["text"].apply(remove_stopwords)

for i in demo_df["text"]:
  if ("https:" in i):
     print (i)

<ipython-input-19-16750eb7d0c1>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  demo_df["text"] = demo_df["text"].apply(remove_stopwords)


In [20]:
# 5. Remove html tags, if any
def remove_html_tags(text):
  pattern=re.compile('<.*?>')
  return pattern.sub("",text)

In [21]:
print ("Before html tag removal")
for i in demo_df["text"]:
  if ("https:" in i):
     print (i)

demo_df["text"] = demo_df["text"].apply(remove_html_tags)

print ("Before html tag removal")
for i in demo_df["text"]:
  if ("https:" in i):
     print (i)

Before html tag removal
Before html tag removal


<ipython-input-21-807fe77d6b78>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  demo_df["text"] = demo_df["text"].apply(remove_html_tags)


In [22]:
# 6. Remove emoji, if any
def remove_emoji(text):
    clean_text=emoji.demojize(text)
    return clean_text

In [23]:
demo_df["text"] = demo_df["text"].apply(remove_emoji)

for i in demo_df["text"]:
  if ("colin" in i):
     print (i)


  count colin marksandspencer xx httpstcohvwkevfskb
115750 love  aestheticcolin  creeperpillar  someday httpstcolfubix5e4y
excellent halloweening marksandspencer countcolin halloween food httpstcolq1csoohdj


<ipython-input-23-fa23eac5574f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  demo_df["text"] = demo_df["text"].apply(remove_emoji)


In [24]:
# 7. Spelling correction
def spelling_correct(text):
    try:
        return TextBlob(text).correct().string
    except:
        return None

In [25]:
demo_df["text"][0:10] = demo_df["text"][0:10].apply(spelling_correct)
demo_df["text"][0:10]

<ipython-input-25-0b1883eac94f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  demo_df["text"][0:10] = demo_df["text"][0:10].apply(spelling_correct)


0    115712  understand  would like  assistwould ne...
1                                  sprintcare  propose
2    sprintcaresent several private messagesone  re...
3    115712 please send us  private messageassistcl...
4                                         sprintcare  
5    115712please send us  private message  gain  d...
6                     sprintcareworst customer service
7    115713saddening  hear please shoot us  do  loo...
8    sprintcare  donna magically change  conductivi...
9    115713  understand  concerns  wed like please ...
Name: text, dtype: object

In [26]:
# 8. Abbrevation expansion

chat_words={
    " AFAIK":"As Far As I Know",
    "AFK": "Away From Keyboard",
    "ASAP":"As Soon As Possible",
    "BTW":"By The Way",
    "B4":"Before",
    "LAMO":"Laugh My A.. Off",
    "FYI":"For your information"
}

In [27]:
def chat_conversion(text):
    new_text=[]
    for w in text.split():
        if w.upper() in chat_words:
            new_text.append(chat_words[w.upper()])
        else:
            new_text.append(w)
    return " ".join(new_text)

In [28]:
demo_df["text"] = demo_df["text"].apply(chat_conversion)
demo_df["text"]

<ipython-input-28-da336e8e0332>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  demo_df["text"] = demo_df["text"].apply(chat_conversion)


0      115712 understand would like assistwould need ...
1                                     sprintcare propose
2      sprintcaresent several private messagesone res...
3      115712 please send us private messageassistcli...
4                                             sprintcare
                             ...                        
195    115769 plans add fast crop modeipad version lr...
196    115775 thanks sharing file well get back updat...
197    adobecare 115776 intermittent file fails relia...
198    115776 unfortunately able reproduce end ill st...
199    adobecare seems pretty widespread httpstcoimom...
Name: text, Length: 200, dtype: object

In [29]:
# 9. Splitting

demo_df["text"][0].split(".")

['115712 understand would like assistwould need get private secured linkassist']

In [30]:
demo_df["text"][0].split()

['115712',
 'understand',
 'would',
 'like',
 'assistwould',
 'need',
 'get',
 'private',
 'secured',
 'linkassist']

In [31]:
word_tokenize(demo_df["text"][0])

['115712',
 'understand',
 'would',
 'like',
 'assistwould',
 'need',
 'get',
 'private',
 'secured',
 'linkassist']

In [32]:
sent_tokenize(demo_df["text"][0])

['115712 understand would like assistwould need get private secured linkassist']

In [33]:
# 10. Stemming

def stemming(text):
    obj=PorterStemmer()
    stem_word=[obj.stem(word) for word in text.split()]
    return stem_word

In [34]:
stemming(demo_df["text"][0])

['115712',
 'understand',
 'would',
 'like',
 'assistwould',
 'need',
 'get',
 'privat',
 'secur',
 'linkassist']

In [35]:
# 11. Lemmetization

def lemmetization(text):
    words=text.split()
    lemmetizer=WordNetLemmatizer()
    lemmetized_word=[lemmetizer.lemmatize(word) for word in words]
    return lemmetized_word

In [36]:
lemmetization(demo_df["text"][0])

['115712',
 'understand',
 'would',
 'like',
 'assistwould',
 'need',
 'get',
 'private',
 'secured',
 'linkassist']

In [42]:
# 12. Encoding - Bag of words

BOW=CountVectorizer()

document_matrix=BOW.fit_transform(demo_df["text"][0:4])

document_matrix

<4x22 sparse matrix of type '<class 'numpy.int64'>'
	with 25 stored elements in Compressed Sparse Row format>

In [44]:
BOW.vocabulary_

{'115712': 0,
 'understand': 18,
 'would': 21,
 'like': 3,
 'assistwould': 1,
 'need': 8,
 'get': 2,
 'private': 10,
 'secured': 13,
 'linkassist': 4,
 'sprintcare': 16,
 'propose': 11,
 'sprintcaresent': 17,
 'several': 15,
 'messagesone': 6,
 'responding': 12,
 'usual': 20,
 'please': 9,
 'send': 14,
 'us': 19,
 'messageassistclick': 5,
 'messagetopprofile': 7}

In [48]:
document_matrix[0].toarray()

array([[1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1]])

In [50]:
document_matrix[1].toarray()

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]])

In [52]:
document_matrix[2].toarray()

array([[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0]])

In [55]:
document_matrix[3].toarray()

array([[1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0]])

In [58]:
# 13. Bigram and Trigram
bigram=CountVectorizer(ngram_range=(2,2))

In [60]:
bigramvocab=bigram.fit_transform(demo_df["text"][0:4])
bigram.vocabulary_

{'115712 understand': 1,
 'understand would': 18,
 'would like': 20,
 'like assistwould': 4,
 'assistwould need': 2,
 'need get': 7,
 'get private': 3,
 'private secured': 11,
 'secured linkassist': 13,
 'sprintcare propose': 16,
 'sprintcaresent several': 17,
 'several private': 15,
 'private messagesone': 10,
 'messagesone responding': 6,
 'responding usual': 12,
 '115712 please': 0,
 'please send': 8,
 'send us': 14,
 'us private': 19,
 'private messageassistclick': 9,
 'messageassistclick messagetopprofile': 5}

In [61]:
trigram=CountVectorizer(ngram_range=(3,3))

In [62]:
trigramvocab=trigram.fit_transform(demo_df["text"][0:4])
trigram.vocabulary_

{'115712 understand would': 1,
 'understand would like': 14,
 'would like assistwould': 16,
 'like assistwould need': 4,
 'assistwould need get': 2,
 'need get private': 6,
 'get private secured': 3,
 'private secured linkassist': 10,
 'sprintcaresent several private': 13,
 'several private messagesone': 12,
 'private messagesone responding': 9,
 'messagesone responding usual': 5,
 '115712 please send': 0,
 'please send us': 7,
 'send us private': 11,
 'us private messageassistclick': 15,
 'private messageassistclick messagetopprofile': 8}

In [65]:
mix=CountVectorizer(ngram_range=(1,2))

In [68]:
mixvocab=mix.fit_transform(demo_df["text"][0:4])
mix.vocabulary_

{'115712': 0,
 'understand': 36,
 'would': 41,
 'like': 7,
 'assistwould': 3,
 'need': 15,
 'get': 5,
 'private': 19,
 'secured': 26,
 'linkassist': 9,
 '115712 understand': 2,
 'understand would': 37,
 'would like': 42,
 'like assistwould': 8,
 'assistwould need': 4,
 'need get': 16,
 'get private': 6,
 'private secured': 22,
 'secured linkassist': 27,
 'sprintcare': 32,
 'propose': 23,
 'sprintcare propose': 33,
 'sprintcaresent': 34,
 'several': 30,
 'messagesone': 12,
 'responding': 24,
 'usual': 40,
 'sprintcaresent several': 35,
 'several private': 31,
 'private messagesone': 21,
 'messagesone responding': 13,
 'responding usual': 25,
 'please': 17,
 'send': 28,
 'us': 38,
 'messageassistclick': 10,
 'messagetopprofile': 14,
 '115712 please': 1,
 'please send': 18,
 'send us': 29,
 'us private': 39,
 'private messageassistclick': 20,
 'messageassistclick messagetopprofile': 11}

In [71]:
# 14. TF IDF
tfidf=TfidfVectorizer()
tfidf.fit_transform(demo_df["text"][0:4]).toarray()

array([[0.2623814 , 0.33279753, 0.33279753, 0.33279753, 0.33279753,
        0.        , 0.        , 0.        , 0.33279753, 0.        ,
        0.21242036, 0.        , 0.        , 0.33279753, 0.        ,
        0.        , 0.        , 0.        , 0.33279753, 0.        ,
        0.        , 0.33279753],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.70710678, 0.        , 0.        , 0.        ,
        0.        , 0.70710678, 0.        , 0.        , 0.        ,
        0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.43003652, 0.        , 0.        , 0.        ,
        0.27448674, 0.        , 0.43003652, 0.        , 0.        ,
        0.43003652, 0.        , 0.43003652, 0.        , 0.        ,
        0.43003652, 0.        ],
       [0.32109252, 0.        , 0.        , 0.        , 0.        ,
        0.4072651

In [72]:
tfidf.get_feature_names_out()

array(['115712', 'assistwould', 'get', 'like', 'linkassist',
       'messageassistclick', 'messagesone', 'messagetopprofile', 'need',
       'please', 'private', 'propose', 'responding', 'secured', 'send',
       'several', 'sprintcare', 'sprintcaresent', 'understand', 'us',
       'usual', 'would'], dtype=object)

In [73]:
tfidf.idf_

array([1.51082562, 1.91629073, 1.91629073, 1.91629073, 1.91629073,
       1.91629073, 1.91629073, 1.91629073, 1.91629073, 1.91629073,
       1.22314355, 1.91629073, 1.91629073, 1.91629073, 1.91629073,
       1.91629073, 1.91629073, 1.91629073, 1.91629073, 1.91629073,
       1.91629073, 1.91629073])

In [101]:
tfidf1=TfidfVectorizer()

text1 = pd.DataFrame({"First" : ["This is a test statement for TFIDF verification ASAP results", "streaming"], "Second" : ["Second sentence", "Stemming"]})
tfidf1.fit_transform(text1["First"])

tfidf1.vocabulary_

{'this': 8,
 'is': 2,
 'test': 6,
 'statement': 4,
 'for': 1,
 'tfidf': 7,
 'verification': 9,
 'asap': 0,
 'results': 3,
 'streaming': 5}

In [102]:
tfidf1.fixed_vocabulary_

False

In [103]:
tfidf1.get_feature_names_out()

array(['asap', 'for', 'is', 'results', 'statement', 'streaming', 'test',
       'tfidf', 'this', 'verification'], dtype=object)

In [104]:
tfidf1.get_stop_words()

In [106]:
tfidf1.idf_

array([1.40546511, 1.40546511, 1.40546511, 1.40546511, 1.40546511,
       1.40546511, 1.40546511, 1.40546511, 1.40546511, 1.40546511])

In [108]:
tfidf.stop_words_

set()